# Final Project – Agentic AI for Financial Research
**Course:** AAI Program  
**Team Members:** [Name 1], [Name 2], [Name 3]  
**GitHub Repository:** [Insert link]

---

## 1. Introduction
- Project overview  
- Goals: Build an agentic AI that researches markets using autonomous agents  
- Deliverables: Notebook (PDF/HTML), GitHub repo, documentation  


## 2. System Architecture
### 2.1 Agent Design
- Orchestrator Agent  
- Data Collection Agents (Yahoo Finance, NewsAPI, SEC)  
- Analysis Agents (Sentiment, Fundamentals, Quant)  
- Decision Agent  
- Critic / Evaluator Agent  
- Memory Module  

### 2.2 Workflow Patterns
- Prompt Chaining  
- Routing  
- Evaluator–Optimizer  

*(Insert diagram of pipeline here)*

In [ ]:
# 3. Setup & Imports
import yfinance as yf # Need to install yfinance package first by running: pip install yfinance
import requests
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
# import openai  # Uncomment if using LLM API

In [ ]:
data = yf.download("AAPL", start="2025-09-18", end="2025-09-18")
print(data.head())

In [ ]:
now = datetime.now()
print("Current date and time:", now.strftime("%Y-%m-%d %H:%M:%S"))

## 4. Agent Functions
We implement the four required capabilities:

1. **Planning** – agent outlines research steps.  
2. **Dynamic Tool Use** – APIs, datasets.  
3. **Self-Reflection** – evaluation of outputs.  
4. **Learning Across Runs** – simple memory store.

In [ ]:
# Example: Orchestrator planning steps for a ticker
def plan_research(symbol):
    steps = [
        f"Fetch price history for {symbol}",
        f"Fetch latest news for {symbol}",
        "Run sentiment analysis on news",
        "Compute technical signals",
        "Generate investment recommendation",
        "Critique and refine output"
    ]
    return steps

plan_research("AAPL")

## 5. Workflow Pattern 1 – Prompt Chaining
Pipeline: News → Preprocess → Classify → Extract → Summarize

In [ ]:
# Example: Fetch and preprocess news
news_sample = [
    {"title": "Apple beats earnings expectations", "text": "Apple reported higher than expected Q2 earnings..."}
]

def preprocess_news(news):
    return [n["text"].lower() for n in news]

processed = preprocess_news(news_sample)
processed

In [ ]:
# Sentiment Classification (placeholder)
def classify_sentiment(text):
    return {"sentiment": "positive", "score": 0.8}

classified = classify_sentiment(processed[0])
classified

In [ ]:
# Entity extraction (placeholder)
def extract_entities(text):
    return ["Apple", "Q2 earnings"]

entities = extract_entities(processed[0])
entities

In [ ]:
# Summarization (placeholder)
def summarize(text):
    return "Apple beat Q2 earnings expectations, likely positive market reaction."

summary = summarize(processed[0])
summary

## 6. Workflow Pattern 2 – Routing
Route input to correct specialized agent.

In [ ]:
def router(input_type, data):
    if input_type == "news":
        return classify_sentiment(data)
    elif input_type == "price":
        return "Quant Agent output"
    elif input_type == "filing":
        return "Fundamentals Agent output"
    else:
        return "Unknown input type"

router("news", "Apple reports record revenue")

## 7. Workflow Pattern 3 – Evaluator–Optimizer
Generate analysis → Evaluate → Refine

In [ ]:
# Simple evaluator-optimizer loop
def generate_analysis(symbol, signals):
    return f"Analysis for {symbol}: Buy based on signals {signals}"

def evaluate_analysis(analysis):
    if "Buy" in analysis:
        return "Good, but lacks risk discussion"
    return "Needs improvement"

def refine_analysis(analysis, feedback):
    return analysis + f" | Refined: {feedback}"

analysis = generate_analysis("AAPL", {"sentiment": "positive", "quant": "bullish"})
feedback = evaluate_analysis(analysis)
refined = refine_analysis(analysis, feedback)
refined

## 8. Demonstration – End-to-End Run
- Choose a stock symbol (e.g., AAPL, TSLA, AMZN)  
- Run through all agents and workflows  
- Show final investment recommendation with rationale  

In [ ]:
symbol = "AAPL"

steps = plan_research(symbol)
print("Planned steps:", steps)

# Price data
price_data = yf.download(symbol, period="6mo", interval="1d")
price_data["Close"].plot(title=f"{symbol} Price Trend")

# Run agents (stub example)
news_summary = summarize("Apple reported strong iPhone sales this quarter.")
sentiment = classify_sentiment("Apple reported strong iPhone sales this quarter.")
decision = generate_analysis(symbol, {"sentiment": sentiment, "quant": "bullish"})
final_output = refine_analysis(decision, evaluate_analysis(decision))

print("Final Investment Thesis:", final_output)

## 9. Evaluation & Iteration
- Example of system critiquing itself and re-running an agent  
- Show before/after outputs  

## 10. Conclusion
- What worked well  
- Challenges  
- Future improvements (better data sources, advanced models, memory persistence)  

In [ ]:
import numpy as np
#from transformers import pipeline

In [ ]:
import numpy as np

# If numpy is not installed, install it using pip
!pip install numpy


In [ ]:
pip install numpy

In [ ]:
import numpy as np
import pandas as pd
#from transformers import pipeline

In [ ]:
from transformers import pipeline

In [ ]:
text = """Dear Amazon, last week I ordered an Optimus Prime action figure
 from your online store in Germany. Unfortunately, when I opened the package,
 I discovered to my horror that I had been sent an action figure of Megatron
 instead! As a lifelong enemy of the Decepticons, I hope you can understand my
 dilemma. To resolve the issue, I demand an exchange of Megatron for the
 Optimus Prime figure I ordered. Enclosed are copies of my records concerning
 this purchase. I expect to hear from you soon. Sincerely, Bumblebee."""

In [ ]:
classifier = pipeline("text-classification")
outputs = classifier(text)
pd.DataFrame(outputs)

In [ ]:
ner_tagger = pipeline("ner", aggregation_strategy="simple")
outputs = ner_tagger(text)
pd.DataFrame(outputs)

In [ ]:
reader = pipeline("question-answering")
question = "What does the customer want?"
outputs = reader(question=question, context=text)
pd.DataFrame([outputs])

In [ ]:
summarizer = pipeline("summarization")
outputs = summarizer(text, max_length=45, clean_up_tokenization_spaces=True)
print(outputs[0]['summary_text'])

In [ ]:
from transformers import pipeline
fill_mask = pipeline("fill-mask", model="metythorn/khmer-xlm-roberta-base")

In [ ]:
#!pip install sentencepiece
result = fill_mask("ខ្ញុំចង់<mask>ភាសាខ្មែរ")
print(result)

In [ ]:
from transformers import pipeline

nlp = pipeline("fill-mask", model="metythorn/khmer-xlm-roberta-base")
nlp("ខ្ញុំស្រលាញ់ <mask>។")  # Example Khmer sentence with a masked token


In [ ]:
translator = pipeline("translation_en_to_de",
model="Helsinki-NLP/opus-mt-en-de")
outputs = translator(text, clean_up_tokenization_spaces=True, min_length=100)
print(outputs[0]['translation_text'])

In [ ]:
 generator = pipeline("text-generation")
 response = "Dear Bumblebee, I am sorry to hear that your order was mixed up."
 prompt = text + "\n\nCustomer service response:\n" + response
 outputs = generator(prompt, max_length=200)
 print(outputs[0]['generated_text'])

In [ ]:
from transformers import AutoTokenizer

In [ ]:
!pip install BertModel

In [ ]:
# The error occurs because 'BertModel' is not a standalone pip package.
# To use BertModel, install the 'transformers' library:
!pip install transformers

# Then import BertModel from transformers:
from transformers import BertModel

In [ ]:
from transformers import BertModel

# The error occurs because 'BertModel' is not a pip-installable package.
# To use BertModel, install the 'transformers' library:
# !pip install transformers

# Then import BertModel from transformers:

In [ ]:
from transformers import AutoTokenizer
from transformers import BertModel

%load_ext autoreload
%autoreload 2



In [ ]:
#from bertviz.neuron_view import show

# To resolve the error, install bertviz first:
!pip install bertviz

# Then you can import show from bertviz.neuron_view:

In [ ]:
from bertviz.neuron_view import show
%load_ext autoreload
%autoreload 2


In [1]:
from bertviz.neuron_view import show

ModuleNotFoundError: No module named 'bertviz'